In [1]:
# hidden num remain unchange
import os
import numpy as np
from torch import optim
from pretreat import *
%matplotlib inline
np.random.seed(0)
training = get_pretreated_training_data()
old_input_dict = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'cabine_n', 'IsAlone']
new_input_dict = old_input_dict + ['sp', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']
loader = get_dataloader(training, new_input_dict, bs=32)
model = MLP(input_dim = len(new_input_dict), hidden_dims=[25, 50, 100, 100, 100, 100,]).cuda()

/home/miniconda/envs/tftorch/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2]:
import torch
old = torch.load('/home/workspace/util/surgery/titanic_demo/runs/May24_08-22-52_1d7a14f7cd78old_network with old input from scratch/oldnetwork_oldinput_from_scratch_bs32.ckpt')

In [3]:
layer = 0
w = "layers.{}.weight".format(layer)
b = "layers.{}.bias".format(layer)
w1 = old['para'][w]
b1 = old['para'][b]
# 构建input和weight的map
inwmap = {}
for i, f in enumerate(old['input']):
    inwmap[f] = w1[:, i]

In [4]:
for i, (name, g) in enumerate(model.named_parameters()):
    if i == 0:
        print(name, g[:5, :])

layers.0.weight tensor([[ 0.0712,  0.1711,  0.0286,  0.2174,  0.1111, -0.2466,  0.1536, -0.1086,
         -0.0978,  0.0979,  0.2072,  0.0763,  0.1197],
        [-0.0311, -0.1523, -0.0476, -0.2222,  0.0586, -0.1493, -0.1335,  0.0629,
         -0.0462,  0.1925,  0.2646,  0.2376,  0.0336],
        [ 0.1049,  0.2172, -0.0644,  0.0820,  0.1205,  0.0026,  0.0719, -0.0132,
         -0.1695, -0.2480, -0.0885, -0.0923,  0.1365],
        [-0.0027,  0.2314, -0.2693,  0.2409,  0.2123, -0.2610,  0.2544, -0.2194,
         -0.0261,  0.0026,  0.2054,  0.2336,  0.1701],
        [-0.1354,  0.0457,  0.0080,  0.2072,  0.2220,  0.2319,  0.0463,  0.0219,
         -0.0730,  0.2766, -0.1118,  0.1385,  0.0391]], device='cuda:0',
       grad_fn=<SliceBackward>)


In [5]:
# 因为只增加了输入 所以只用对w1进行初始化特殊处理. 
for i, (name, g) in enumerate(model.named_parameters()):
    if i == 0:
        new_g = torch.zeros_like(g)
        for j, f in enumerate(new_input_dict):
            if f in inwmap.keys():
                new_g[:, j] = inwmap[f]
        g.data = new_g
    else:
        g.data = old['para'][name].data

In [6]:
for i, (name, g) in enumerate(model.named_parameters()):
    if i == 0:
        print(name, g[:5, :])

layers.0.weight tensor([[ 0.0085, -0.2214,  0.4244, -0.3489,  0.1652,  0.0302,  0.0053,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.9477,  0.0285,  0.3445, -0.6276,  0.1586, -0.4775, -0.8388,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.4387, -0.4801, -0.1163, -0.3614,  0.4226,  0.1902, -0.0195,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.5647, -0.0992,  0.8484,  0.4323, -0.0145, -0.1077,  0.3306,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.1149,  0.2463, -0.4758, -0.4103, -0.1542, -1.4015, -0.4041,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000]], device='cuda:0',
       grad_fn=<SliceBackward>)


In [7]:
# get surgery之后最初始的acc
sum_acc = 0
for x, y in loader:
    x, y = x.cuda(), y.cuda()
    outputs=model(x)
    _,id=torch.max(outputs.data,1)
    acc=torch.sum(id==y.data)
    sum_acc += acc.item()    
print(sum_acc)

752


/home/miniconda/envs/tftorch/lib/python3.6/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [8]:
lr = 0.001
epoches = 2000 
comment = 'new_network with new input using Surgery'
model_name = 'newnet_newinput_using_surgery_bs32.ckpt'
input_dic = new_input_dict
acc_cur, loss_cur = train(model, loader, comment, model_name, input_dic, )